<a href="https://colab.research.google.com/github/carlos-alves-one/-ML-Zoomcamp-HW-6/blob/main/ML_Zoomcamp_HW_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Learning ZoomCamp - Homework 6
**Author.....: Carlos Manuel de Oliveira Alves**<br>
**Created...: 18/10/2022**

In [2]:
# Import all necessay libraries that we will use in this homework

import re
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Import the library warnings to ignore the warnings from the system
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [3]:
# Locate dataset for the homework
df_url='https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv'
df=pd.read_csv(df_url)